# 主要是一些复杂的动画支持

参考
[Manim Example Gallery](https://manim-example-gallery.netlify.app/src/batchanimationexample)

In [1]:
# 导入
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

Manim Community v0.17.2

In [8]:
%%manim -ql -v WARNING DissipatingPathExample
class DissipatingPathExample(Scene):
    def construct(self):
        a = Dot(RIGHT * 2)
        b = TracedPath(a.get_center, dissipating_time=0.5, stroke_opacity=[0, 1])
        self.add(a, b)
        self.play(a.animate(path_arc=PI / 4).shift(LEFT * 2))
        self.play(a.animate(path_arc=-PI / 4).shift(LEFT * 2))
        self.wait()

In [10]:
%%manim -ql -v WARNING TestScene
def shuffle(string):
    '''随机打乱字符串并返回新的字符串'''
    import random
    char_list = list(string)
    random.shuffle(char_list)
    return ''.join(char_list)

class TestScene(Scene):
    def construct(self):
        # 生成20个字符
        chars = VGroup(*(
            Text(char, color = utils.color.random_color())
            for _ in range(4) for char in shuffle('MANIM')
        ))
        # 网格排列（4行5列）
        chars.arrange_in_grid(rows=4)

        # 用AnimationGroup管理多个FadeIn动画
        self.play(AnimationGroup(
            *(FadeIn(m, shift=DOWN) for m in chars),
            lag_ratio = .3,
        ))
        self.wait()

In [11]:
%%manim -ql -v WARNING Circuit
class Circuit(Scene):
    def construct(self):
        template = TexTemplate()
        template.add_to_preamble(r"\usepackage{circuitikz}")

        c = MathTex(
            r'''\draw 
                (0,2)   to[battery1]      (0,0)
                (0,2)   --                (2,2)
                (2,2)   to[R=\(R_1\)]     (2,0)
                (2,2)   --                (4,2)
                (4,2)   to[rmeter, t=V]   (4,0)
                (4,0)   --                (2,0)
                (2,0)   to[rmeter, t=A]   (0,0);
            ''',
            stroke_width=4,
            fill_opacity=0,
            stroke_opacity=1,
            tex_environment="circuitikz",
            tex_template=template,
        )

        self.play(Write(c, run_time=3))
        self.wait()

In [12]:
%%manim -ql -v WARNING MoveCamera
class MoveCamera(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes(x_range=(-0.1, 4.25), y_range=(-1.5, 1.5), z_range=(-1.5, 1.5), y_length=5, z_length=5)
        func = lambda p: axes.coords_to_point(p, np.exp(complex(0, PI*p)).real, np.exp(complex(0, PI*p)).imag)
        
        curve = ParametricFunction(
            func,
            t_range=(0, 2, 0.1)
        )
        curve_extension = ParametricFunction(
            func,
            t_range=(2, 4, 0.1)
        )

        self.set_camera_orientation(phi=90*DEGREES, theta=0*DEGREES, focal_distance=10000)
        self.add(axes)
        self.play(Create(curve, run_time=2))
        self.wait()
        self.move_camera(phi=130*DEGREES, theta=-60*DEGREES, focal_distance=10000)
        self.wait()
        self.play(Create(curve_extension, run_time=2))
        self.wait()
        self.move_camera(phi=90*DEGREES, theta=-90*DEGREES, focal_distance=10000)
        self.wait()

In [13]:
%%manim -ql -v WARNING RestoreExample
class RestoreExample(Scene):
    def construct(self):
        s = Square()
        s.save_state()
        self.play(FadeIn(s))
        self.play(s.animate.set_color(PURPLE).set_opacity(0.5).shift(2*LEFT).scale(3))
        self.play(s.animate.shift(5*DOWN).rotate(PI/4))
        self.wait()
        self.play(Restore(s), run_time=2)

In [14]:
%%manim -ql -v WARNING TimeUpdater
class TimeUpdater(Scene):
    def construct(self):
        def updater_forth(mobj, dt):
            mobj.rotate_about_origin(dt)
        def updater_back(mobj, dt):
            mobj.rotate_about_origin(-dt)
        line_reference = Line(ORIGIN, LEFT).set_color(WHITE)
        line_moving = Line(ORIGIN, LEFT).set_color(YELLOW)
        line_moving.add_updater(updater_forth)
        self.add(line_reference, line_moving)
        self.wait(2)
        line_moving.remove_updater(updater_forth)
        line_moving.add_updater(updater_back)
        self.wait(2)
        line_moving.remove_updater(updater_back)
        self.wait(0.5)

In [15]:
%%manim -ql -v WARNING CountingScene
from manim import *

class Count(Animation):
    def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
        # Pass number as the mobject of the animation
        super().__init__(number,  **kwargs)
        # Set start and end
        self.start = start
        self.end = end

    def interpolate_mobject(self, alpha: float) -> None:
        # Set value of DecimalNumber according to alpha
        value = self.start + (alpha * (self.end - self.start))
        self.mobject.set_value(value)


class CountingScene(Scene):
    def construct(self):
        # Create Decimal Number and add it to scene
        number = DecimalNumber().set_color(WHITE).scale(5)
        # Add an updater to keep the DecimalNumber centered as its value changes
        number.add_updater(lambda number: number.move_to(ORIGIN))

        self.add(number)

        self.wait()

        # Play the Count Animation to count from 0 to 100 in 4 seconds
        self.play(Count(number, 0, 100), run_time=4, rate_func=linear)

        self.wait()